## Ordinary Least Square
#### This notebook provides a matrix implementation of OLS and explores the properties of BLUE (Best Linear Unbiased Estimate). The subclass also provides a L2-penalty(Ridge regression) in matrix form

#### 1) Test for for multicolinearity, heteroskadesticity, and auto correlation
#### 2) T-test - for individual covariate
#### 3) F-test - for all covariates 
#### 4) L2-regularization Ridge Regression


In [ ]:
from sklearn.datasets import fetch_california_housing as fetch_data
import statsmodels.api as sm
import numpy as np
import pandas as pd
import scipy.stats as st


In [2]:
data = fetch_data()
housing_data = pd.DataFrame(data.data)
housing_data.columns = data.feature_names
targets = data.target
display(housing_data.head())
display(targets)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [3]:
class OLS_model(object):
    
    def __init__(self,y,X,intercept=True):
        data_x = X.copy()
        
        if intercept:
            data_x.insert(0,'intercept',np.ones(len(data_x)))
        else:
            pass
        
        self.columns =data_x.columns
        
        self.X = np.mat(data_x) # stored as matrix
        
        self.b = None
        
        self.y = y.copy()
        
        self.N = self.X.shape[0]
        self.K = self.X.shape[1]
        
    def fit_data(self,output=True):
        
        self.b = np.array(np.dot(np.dot(np.linalg.inv(np.dot(self.X.T,self.X)),self.X.T), np.mat(self.y).T))
        
        #for i in range(self.K):
        #    print(f'vairable {self.columns[i]} coef is {self.b[i][0]}')
        return self.b
                          
    def predict_y(self,X_pred):
        y_hat =np.dot(X_pred,self.b) 
        return y_hat
                          
    def t_test(self,alpha=0.05):
        X_XT_inv = np.linalg.inv(np.dot(self.X.T,self.X))
        y_hat =np.array(np.dot(self.X,self.b).T)[0]
        sigma_hat = np.sum((y_hat-self.y)**2)/(self.N-self.K)
        #print(X_XT_inv)
        for i in range(self.K-1):
            sk = X_XT_inv[i,i]
            #print( f'sk value is{sk}')
            Tk = self.b[i][0]/np.sqrt(sigma_hat*sk)
            #print(f'Tk value is{Tk}')
            if abs(Tk)>abs(st.t.ppf(alpha/2,df = self.N-self.K)):
                print(f'reject H0. T of {i} is {Tk}')
            else:
                print(f'can\'t reject H0. T of {i} is {Tk}\n' )
        return
    
    def R_square(self):
        y_mean = np.mean(self.y)
        y_hat = np.array(np.dot(self.X,self.b).T)[0]
        TSS = np.sum(np.square(self.y-y_mean))
        RSS = np.sum(np.square(self.y-y_hat))
        self.R_2 = 1-RSS/TSS
        return self.R_2
    
    def f_test(self,alpha=0.05):
        r2 = self.R_square()
        F = r2/(self.K-1)/((1-r2)/(self.N-self.K))
        if abs(F)>abs(st.f.ppf(1-alpha,dfn=self.K-1,dfd=self.N-self.K)):
            print(f'F of function is {F}, reject H0')
        else:
            print(f'F of function is {F}, can\'t reject H0')
        return
    
    def data_corr(self):
        return pd.DataFrame(self.X[:,1:]).corr() # exclude intercept
    
    def VIF_cal(self,index):
        X_k = np.array(self.X[:,index].T)
        X_rest =self.X[:,[i for i in range(index)] + [j for j in range(index+1,self.K)]] 
        
        vif_model = OLS_model(X_k, pd.DataFrame(X_rest),intercept=False)
        vif_model.fit_data()
        R_k_2 = vif_model.R_square()
        VIF_k = 1/(1-R_k_2)
        return VIF_k
                
    def VIF_test(self):
        for k in range(1,self.K):
            VIF_k = self.VIF_cal(k)
            pritn(f'variable {i}\'s VIF is {VIF_k}' )
            
    def cond_num(self):
        XT = self.X.T
        X_2 = np.dot(XT[[i for i in range(1,self.K)],:], self.X[:,[i for i in range(1,self.K)]])
        eig_values,eig_vector = np.linalg.eig(X_2)
        conditional_num = np.sqrt(eig_values[0]/eig_values[-1])
        #print(f'The conditional number is {cond_num}')
        return conditional_num
    
    def ridge_fit(self,k=0.1):
        XT= self.X.T
        ridge_b = np.array(np.dot(np.dot(np.linalg.inv(np.dot(XT,self.X)+k*np.mat(self.K)),XT),self.y))
        print(f'The shape of ridge_B is {ridge_b.shape}')
        for i in range(self.K):
            print(f'variable {i} coef is {ridge_b[0][i]}')
        
        return
        

In [4]:
ols_m = OLS_model(targets,housing_data)

In [5]:
ols_m.data_corr()

,0,1,2,3,4,5,6,7
0,1.000000,-0.119034,0.326895,-0.062040,0.004834,0.018766,-0.079809,-0.015176
1,-0.119034,1.000000,-0.153277,-0.077747,-0.296244,0.013191,0.011173,-0.108197
2,0.326895,-0.153277,1.000000,0.847621,-0.072213,-0.004852,0.106389,-0.027540
3,-0.062040,-0.077747,0.847621,1.000000,-0.066197,-0.006181,0.069721,0.013344
4,0.004834,-0.296244,-0.072213,-0.066197,1.000000,0.069863,-0.108785,0.099773
5,0.018766,0.013191,-0.004852,-0.006181,0.069863,1.000000,0.002366,0.002476
6,-0.079809,0.011173,0.106389,0.069721,-0.108785,0.002366,1.000000,-0.924664
7,-0.015176,-0.108197,-0.027540,0.013344,0.099773,0.002476,-0.924664,1.000000


In [6]:
ols_m.cond_num()

10310.017246909201

In [7]:
# since we found that variable 2 and 3 are correlated, 6 and 7 are correlated
# we keep one in each pair   
housing_data = housing_data[['MedInc','HouseAge','AveRooms','Population','AveOccup','Latitude']]
housing_data.head()

,MedInc,HouseAge,AveRooms,Population,AveOccup,Latitude
0,8.3252,41.0,6.984127,322.0,2.555556,37.88
1,8.3014,21.0,6.238137,2401.0,2.109842,37.86
2,7.2574,52.0,8.288136,496.0,2.802260,37.85
3,5.6431,52.0,5.817352,558.0,2.547945,37.85
4,3.8462,52.0,6.281853,565.0,2.181467,37.85


In [8]:
housing_data.corr()

,MedInc,HouseAge,AveRooms,Population,AveOccup,Latitude
MedInc,1.000000,-0.119034,0.326895,0.004834,0.018766,-0.079809
HouseAge,-0.119034,1.000000,-0.153277,-0.296244,0.013191,0.011173
AveRooms,0.326895,-0.153277,1.000000,-0.072213,-0.004852,0.106389
Population,0.004834,-0.296244,-0.072213,1.000000,0.069863,-0.108785
AveOccup,0.018766,0.013191,-0.004852,0.069863,1.000000,0.002366
Latitude,-0.079809,0.011173,0.106389,-0.108785,0.002366,1.000000


In [9]:
ols_m=OLS_model(targets,housing_data)
ols_m.fit_data()
ols_m.t_test(0.05)
ols_m.f_test(0.05)

reject H0. T of 0 is 15.739114387152433
reject H0. T of 1 is 139.6404385552682
reject H0. T of 2 is 37.246561597719456
reject H0. T of 3 is -8.409713632171224
reject H0. T of 4 is 3.797169664074015
reject H0. T of 5 is -8.50938020001479
F of function is 3738.7344946400617, reject H0


In [10]:
#compare with statsmodels
built_in_ols = sm.OLS(targets,sm.add_constant(housing_data))
result = built_in_ols.fit()
display(result.params)
display(result.summary())

const         1.549259
MedInc        0.436954
HouseAge      0.017581
AveRooms     -0.020472
Population    0.000020
AveOccup     -0.004572
Latitude     -0.044304
dtype: float64

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     3739.
Date:                Wed, 13 Oct 2021   Prob (F-statistic):               0.00
Time:                        21:27:00   Log-Likelihood:                -24648.
No. Observations:               20640   AIC:                         4.931e+04
Df Residuals:                   20633   BIC:                         4.937e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5493      0.098     15.739      0.000       1.356       1.742
MedInc         0.4370      0.003    139.640      0.000       0.431       0.443
HouseAge       0.0176      0.000     37.247      0.000       0.017       0.019
AveRooms      -0.0205      0.002     -8.410      0.000      -0.025      -0.016
Population  1.983e-05   5.22e-06      3.797      0.000    9.59e-06    3.01e-05
AveOccup      -0.0046      0.001     -8.509      0.000      -0.006      -0.004
Latitude      -0.0443      0.003    -16.710      0.000      -0.050      -0.039
==============================================================================
Omnibus:                     4379.455   Durbin-Watson:                   0.807
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11132.049
Skew:                           1.162   Prob(JB):                         0.00
Kurtosis:                       5.747   Cond. No.                     3.22e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
# we see that the coefficients are equal , t-statistics are equal, and the f-statistic is equal.

In [12]:
ols_m.ridge_fit()

The shape of ridge_B is (1, 7)
variable 0 coef is 1.5295821869541477
variable 1 coef is 0.437069059438469
variable 2 coef is 0.017600006889152628
variable 3 coef is -0.020478545785407753
variable 4 coef is 2.0068917875831756e-05
variable 5 coef is -0.0045730470538956
variable 6 coef is -0.04378896063672226


In [13]:
# reload all the variables including correlated ones
orig_housing_data = fetch_data()
housing_data = pd.DataFrame(orig_housing_data.data)
housing_data.columns =orig_housing_data.feature_names

In [14]:
ols_m2 = OLS_model(targets,housing_data)
# exclude intercept column
for i in range(1,ols_m2.K):
    print(f"the VIF of variable{i} is {ols_m2.VIF_cal(i)}")
print(f"the conditional number is {ols_m2.cond_num()}")

the VIF of variable1 is 2.5012945125415973
the VIF of variable2 is 1.2412541182182886
the VIF of variable3 is 8.342785615374497
the VIF of variable4 is 6.994994771360953
the VIF of variable5 is 1.138125081833602
the VIF of variable6 is 1.0083244489804453
the VIF of variable7 is 9.297624369314345
the VIF of variable8 is 8.96226347381992
the conditional number is 10310.017246909201


In [15]:
# hetegoskadesticity issue
# White Test:
# epsilon ^2  = X + X^2 + cross-product of X 
# WT(k-1) = nR^2 ~ chi-square dist
from statsmodels.stats.diagnostic import het_white
# use np.reshape to modify (n,) to (n,1) so that it matches matrix (n,k)*(k,1)
residuals = np.reshape(targets,(targets.shape[0],1)) - np.dot(ols_m.X,ols_m.b)
white_stat = het_white(residuals,ols_m.X)
white_stat

(2286.66049078056, 0.0, 95.11367432672631, 0.0)

In [16]:
targets=np.reshape(targets,(targets.shape[0],1))

In [17]:
# GLS
ols_resid = sm.OLS(targets,housing_data).fit().resid
res_fit = sm.OLS(ols_resid[1:].values,ols_resid[:-1].values).fit() # fit an AR(1) process
rho= res_fit.params
rho

array([0.54541885])

In [18]:
from scipy.linalg import toeplitz
n = len(housing_data)
order = toeplitz(np.arange(n))
sigma_matrix = rho**order
#display(sigma_matrix)
display(sigma_matrix.shape)
# ===> 20640 X 20640  to show a smaller example use a subset of the data
order = toeplitz(np.arange(100))
sigma_matrix = rho**order

(20640, 20640)

In [19]:
gls_model = sm.GLS(targets[:100],housing_data[:100],sigma=sigma_matrix)
gls_results= gls_model.fit()
print(gls_results.summary())

                                 GLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.687
Model:                            GLS   Adj. R-squared (uncentered):              0.660
Method:                 Least Squares   F-statistic:                              25.27
Date:                Wed, 13 Oct 2021   Prob (F-statistic):                    3.72e-20
Time:                        21:27:22   Log-Likelihood:                         -76.211
No. Observations:                 100   AIC:                                      168.4
Df Residuals:                      92   BIC:                                      189.3
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------